In [1]:
# imports

import pyfastx
import psa
import os
import pandas as pd
from mjol import gan as mgan
from tqdm import tqdm

In [16]:
in_base = "/ccb/salz4-3/hji20/hg002-q100-annotation/results/6_add_copies"
out_base = "/ccb/salz4-3/hji20/hg002-q100-annotation/results/analysis/broken"
MIN_PIDENT = 80.0

In [3]:
def is_coding_gene(f):
    for tx in f.children:
        for c in tx.children:
            if c.feature_type == "CDS":
                return True
    return False

### Load MANE (v1.4)

In [4]:
fn = "/ccb/salz4-3/hji20/hg002-q100-annotation/data/MANE.GRCh38.v1.4.refseq_genomic.mane_sel.noAlt.noFix.gff"

df = pd.read_csv(fn, sep='\t', header=None)
len(df)

HDR = [
    'chr', 'src', 'feature_type', 'start', 
    'end', 'score', 'strand', 'frame', 'attributes'
]
df.columns = HDR

In [5]:
for name, sub_df in df.groupby('feature_type'):
    if name == "CDS":
        mane_cds_df = sub_df

In [6]:
mane_genes_tbl = dict()
for _, row in mane_cds_df.iterrows():
    temp = row['attributes'].split(';')
    att_tbl = {x : y for x, y in [x.split('=') for x in temp]}
    mane_genes_tbl[att_tbl['gene']] = att_tbl['Name']

In [7]:
mane_fa = pyfastx.Fasta("/ccb/salz4-3/hji20/hg002-q100-annotation/data/MANE.GRCh38.v1.4.refseq_protein.faa")

### Examine MAT MANE transcripts

In [8]:
wkdir = os.path.join(in_base, 'mat_loff')

In [9]:
qry_fa = pyfastx.Fasta(os.path.join(wkdir, 'fmted.modified.prot.fa'))

In [10]:
in_fn = os.path.join(wkdir, 'fmted.modified.gff')
mat_gan = mgan.GAn(
    file_name = in_fn,
    file_fmt = 'gff'
)
mat_gan.build_db()

In [11]:
mane_sel_tinfos = dict()
mane_sel_g2t = dict()
gid2gname = dict()
for _, f in mat_gan.features.items():
    if f.feature_type == "gene":
        if is_coding_gene(f):
            if f.aid:
                gene_id = f.aid
                if 'gene_name' not in f.attributes:
                    print(f'gene name attribute not available for {f}')
                    continue
                gid2gname[gene_id] = f.attributes['gene_name']
                for child in f.children:
                    if not child.aid:
                        print(f'attribute ID not available for {child}')
                        continue
                    if 'tag' in child.attributes and \
                        child.attributes['tag'] == "MANE Select":
                        mane_sel_g2t[gene_id] = child.aid
                        assert 'valid_ORF' in child.attributes
                        if child.attributes['valid_ORF'] == "False":
                            mane_sel_tinfos[child.aid] = False
                        else:
                            mane_sel_tinfos[child.aid] = True

In [12]:
print(f'{len([x for x in mane_sel_tinfos.values() if not x])} genes w/ broken MANE')

84 genes w/ broken MANE


In [13]:
mane_sel_tids2align = [x for x, y in mane_sel_tinfos.items() if x]
print(f'{len(mane_sel_tids2align)} genes to further analyze for potential truncation')

19134 genes to further analyze for potential truncation


In [ ]:
aligned = 0
truncated = []
for gene_id, tid in tqdm(mane_sel_g2t.items(), total=len(mane_sel_g2t)):
    gene_name = gid2gname[gene_id]
    if not mane_sel_tinfos[tid]: # no need to align
        continue
    if gene_name not in mane_genes_tbl:
        continue
    mane_pid = mane_genes_tbl[gene_name]
    aligned += 1
    qseq = qry_fa[tid].seq
    sseq = mane_fa[mane_pid].seq
    aln = psa.needle(moltype='prot', qseq=qseq, sseq=sseq)
    if float(aln.pidentity) < MIN_PIDENT:
        # technically, we didn't need to store tid here
        truncated.append((gene_id, tid, aln.pidentity))

100%|██████████| 19134/19134 [07:32<00:00, 42.30it/s]  


In [19]:
out_fn = os.path.join(out_base, 'mat_mane_status.tsv')
out_fh = open(out_fn, 'w')
out_fh.write(f'gene_id\tgene_name\ttranscript_id\tbroken\tpident\n')

truncated_ginfos = { x : z for x, y, z in truncated }
for gene_id, tid in mane_sel_g2t.items():
    gene_name = gid2gname[gene_id]
    if not mane_sel_tinfos[tid]:
        out_fh.write(f'{gene_id}\t{gene_name}\t{tid}\tY\tNA\n')
        continue
    if gene_name not in mane_genes_tbl:
        out_fh.write(f'{gene_id}\t{gene_name}\t{tid}\tNA\tNA\n')
        continue
    if gene_id in truncated_ginfos:
        pident = truncated_ginfos[gene_id]
        out_fh.write(f'{gene_id}\t{gene_name}\t{tid}\tY\t{pident}\n')
    else:
        out_fh.write(f'{gene_id}\t{gene_name}\t{tid}\tN\tNA\n')
out_fh.close()

### Examine PAT MANE transcripts

In [20]:
wkdir = os.path.join(in_base, 'pat_loff')

In [21]:
qry_fa = pyfastx.Fasta(os.path.join(wkdir, 'fmted.modified.prot.fa'))

In [22]:
in_fn = os.path.join(wkdir, 'fmted.modified.gff')
pat_gan = mgan.GAn(
    file_name = in_fn,
    file_fmt = 'gff'
)
pat_gan.build_db()

In [25]:
mane_sel_tinfos = dict()
mane_sel_g2t = dict()
gid2gname = dict()
for _, f in pat_gan.features.items():
    if f.feature_type == "gene":
        if is_coding_gene(f):
            if f.aid:
                gene_id = f.aid
                if 'gene_name' not in f.attributes:
                    print(f'gene name attribute not available for {f}')
                    continue
                gid2gname[gene_id] = f.attributes['gene_name']
                for child in f.children:
                    if not child.aid:
                        print(f'attribute ID not available for {child}')
                        continue
                    if 'tag' in child.attributes and \
                        child.attributes['tag'] == "MANE Select":
                        mane_sel_g2t[gene_id] = child.aid
                        assert 'valid_ORF' in child.attributes
                        if child.attributes['valid_ORF'] == "False":
                            mane_sel_tinfos[child.aid] = False
                        else:
                            mane_sel_tinfos[child.aid] = True

In [26]:
print(f'{len([x for x in mane_sel_tinfos.values() if not x])} genes w/ broken MANE')

75 genes w/ broken MANE


In [27]:
mane_sel_tids2align = [x for x, y in mane_sel_tinfos.items() if x]
print(f'{len(mane_sel_tids2align)} genes to further analyze for potential truncation')

18381 genes to further analyze for potential truncation


In [28]:
aligned = 0
truncated = []
for gene_id, tid in tqdm(mane_sel_g2t.items(), total=len(mane_sel_g2t)):
    gene_name = gid2gname[gene_id]
    if not mane_sel_tinfos[tid]: # no need to align
        continue
    if gene_name not in mane_genes_tbl:
        continue
    mane_pid = mane_genes_tbl[gene_name]
    aligned += 1
    qseq = qry_fa[tid].seq
    sseq = mane_fa[mane_pid].seq
    aln = psa.needle(moltype='prot', qseq=qseq, sseq=sseq)
    if float(aln.pidentity) < MIN_PIDENT:
        # technically, we didn't need to store tid here
        truncated.append((gene_id, tid, aln.pidentity))

100%|██████████| 18381/18381 [07:03<00:00, 43.44it/s]  


In [29]:
out_fn = os.path.join(out_base, 'pat_mane_status.tsv')
out_fh = open(out_fn, 'w')
out_fh.write(f'gene_id\tgene_name\ttranscript_id\tbroken\tpident\n')

truncated_ginfos = { x : z for x, y, z in truncated }
for gene_id, tid in mane_sel_g2t.items():
    gene_name = gid2gname[gene_id]
    if not mane_sel_tinfos[tid]:
        out_fh.write(f'{gene_id}\t{gene_name}\t{tid}\tY\tNA\n')
        continue
    if gene_name not in mane_genes_tbl:
        out_fh.write(f'{gene_id}\t{gene_name}\t{tid}\tNA\tNA\n')
        continue
    if gene_id in truncated_ginfos:
        pident = truncated_ginfos[gene_id]
        out_fh.write(f'{gene_id}\t{gene_name}\t{tid}\tY\t{pident}\n')
    else:
        out_fh.write(f'{gene_id}\t{gene_name}\t{tid}\tN\tNA\n')
out_fh.close()